In [65]:
'''
1. In the following Jupyter file we used openai text-embedding-text-small-3 model for generating the embeddings and llama3.1-70B
    for the retrieving process.

2. We also created a custom Embedding Class in llamaindex to be passed in the settings which uses the github model and generate
    the embeddings for the documents given.

3. The main thing I understand with the notebook is that you can use any embeddings model with any other RAG model like below.
    I used OpenAI embeddings to generate the embeddings but used the llama3.1-70B model for the retriving process which means.
    It doesnt matter about the model, The only thing matter is the embedding model used to generate the embeddings for the 
    documents that should be the same for the user's query embeddings. It doesnt matter the other model like llama3.1

Example: - 
'''

"\n1. In the following Jupyter file we used openai text-embedding-text-small-3 model for generating the embeddings and llama3.1-70B\n    for the retrieving process.\n\n2. We also created a custom Embedding Class in llamaindex to be passed in the settings which uses the github model and generate\n    the embeddings for the documents given.\n\n3. The main thing I understand with the notebook is that you can use any embeddings model with any other RAG model like below.\n    I used OpenAI embeddings to generate the embeddings but used the llama3.1-70B model for the retriving process which means.\n    It doesnt matter about the model, The only thing matter is the embedding model used to generate the embeddings for the \n    documents that should be the same for the user's query embeddings. It doesnt matter the other model like llama3.1\n\nExample: - \n"

In [31]:
from llama_index.core import Settings, VectorStoreIndex, StorageContext, SimpleDirectoryReader, load_index_from_storage
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import PromptTemplate
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine
from openai import OpenAI
from llama_index.llms.groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
reader = SimpleDirectoryReader('./', required_exts=['.txt'])
docs = reader.load_data()

In [37]:
from typing import Any, List
from openai import OpenAI
from llama_index.core.bridge.pydantic import PrivateAttr
from llama_index.core.embeddings import BaseEmbedding

In [57]:
class OpenAIEmbeddings_github_models(BaseEmbedding):
    _client: OpenAI = PrivateAttr()
    _model_name: str = PrivateAttr()

    def __init__(
        self,
        api_key: str = os.environ["GITHUB_TOKEN"],
        endpoint: str = "https://models.inference.ai.azure.com",
        model_name: str = "text-embedding-3-small",
        **kwargs: Any,
    ) -> None:
        super().__init__(**kwargs)
        self._client = OpenAI(
            base_url=endpoint,
            api_key=api_key,
        )
        self._model_name = model_name

    @classmethod
    def class_name(cls) -> str:
        return "openai"

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)

    def _get_query_embedding(self, query: str) -> List[float]:
        response = self._client.embeddings.create(
            input=[query],
            model=self._model_name
        )
        # Assuming response.data is a list of objects with 'embedding' attribute
        embeddings = [item.embedding for item in response.data]
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        response = self._client.embeddings.create(
            input=[text],
            model=self._model_name
        )
        embeddings = [item.embedding for item in response.data]
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        response = self._client.embeddings.create(
            input=texts,
            model=self._model_name
        )
        embeddings = [item.embedding for item in response.data]
        return embeddings

    def _print_embedding_dimensions(self, embedding: List[float]) -> None:
        print(f"Embedding dimensions: {len(embedding)}")

In [59]:
embed = OpenAIEmbeddings_github_models()

embeddings = embed.get_text_embedding('My name is vedant rajpurohit')

embed._print_embedding_dimensions(embeddings)

Embedding dimensions: 1536


In [54]:
Settings.llm = Groq(model="llama-3.1-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))
Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
Settings.embed_model = OpenAIEmbeddings_github_models()

In [55]:
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    index = VectorStoreIndex.from_documents(
        docs,
        transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)]
    )
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

In [56]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is vedant rajpurohit?")
print(response)

Vedant Rajpurohit is a Junior AI Engineer with a Bachelor of Technology degree in Computer Science Specialized with Machine learning and AI. He has completed various internships and worked as a teaching professional in the IT field. He is seeking a career in data science and Generative AI in a challenging environment where he can utilize his technical skills towards the development and implementation of new ideas and contribute to the growth of the organization.
